In [0]:
# %pip install "typing_extensions==4.5.0" "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2" "langchain"

# Imports

In [0]:
import torch
from transformers import pipeline

# langchain
from langchain.llms import HuggingFacePipeline

# langchain chains
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

# Models

In [0]:
architecture = "databricks/dolly-v2-7b"

In [0]:
dolly = pipeline(
    model=architecture,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    return_full_text=True,
)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [0]:
hf_pipeline = HuggingFacePipeline(pipeline=dolly)

# LLMChain

In [0]:
product = "Queen Size Sheet Set"

In [0]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [0]:
chain = LLMChain(llm=hf_pipeline, prompt=prompt)
chain.run(product)

Out[17]: '\nQueen Size Sheet Set Company'

# SimpleSequentialChain
- Single Input/Output

<span>Input -> <b>o-o-o</b> -> Output of Chain 1 = Input of Chain 2 -> <b>o-o-o</b> -> Output</span>

In [0]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

In [0]:
# chain 1
chain_one = LLMChain(llm=hf_pipeline, prompt=first_prompt)

# chain 2
chain_two = LLMChain(llm=hf_pipeline, prompt=second_prompt)

In [0]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...

Queen Size Sheet Set Company best describes a company that makes queen size sheet sets.

A Queen Size Sheet Set Company makes queen size sheet sets.  A queen size sheet set is a set of sheets, pillowcases, and a quilt set all in one package for a total of 504 pieces of linen and cotton that are sized to fit a King or Queen size bed.  The sets can be completely customizable, with the customer able to pick the colors of sheets, thread count, weight of the fabric, and size of the pillowcases and quilt.  The sets are often purchased by customers who have recently moved into a new house or apartment and want to ensure they have all the right pieces to completely decorate their new space with a theme that matches their past life.

> Finished chain.
Out[24]: '\nA Queen Size Sheet Set Company makes queen size sheet sets.  A queen size sheet set is a set of sheets, pillowcases, and a quilt set all in one package for a total of 504 pieces of linen

# SequentialChain
- Multiple Inputs/Outputs

<span>Input -> <b>o-o-o</b> -> Output of Chain 1 = Input of Chain 2 -> <b>o-o-o</b> -> Output of Chain 2 = Input of Chain 4</span>
<br>
<span>Input -> <b>o-o-o</b> -> Output of Chain 3 = Input of Chain 4</span> 

<span>Chain 2 and Chain 4 Outputs -> Input -> <b>o-o-o</b> -> Output of Chain 4</span>

In [0]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
)

# prompt template 2: summarize the english
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

In [0]:
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=hf_pipeline, prompt=first_prompt, output_key="English_Review")

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=hf_pipeline, prompt=second_prompt, output_key="summary")

# chain 3: input= Review and output= language
chain_three = LLMChain(llm=hf_pipeline, prompt=third_prompt, output_key="language")

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(
    llm=hf_pipeline, prompt=fourth_prompt, output_key="followup_message"
)

In [0]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

In [0]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.
Out[33]: {'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "\nThe review is kind of long, but basically compares their service to Translate.com. This is how I compared them:\n\nThey both offer translation services.\nTranslate.com requires you to pay per word, while others don't. I think this is a reasonable requirement for a translation service.\nTranslate.com offers more services, such as automatic correction and rewording.\nThey also both offer computer translations. I think this is also a reasonable requirement for a translation service.\n\nI think this is a fair comparison of the 2 services.",
 'summary': '\nThe reviewer summarizes Translate.com in 1 sentence by saying that they have a reasonable requirement for a translation service: require you to pay per word.',
 'followup_m

# Router Chain

In [0]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [0]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [0]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=hf_pipeline, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [0]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=hf_pipeline, prompt=default_prompt)

In [0]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [0]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(hf_pipeline, router_prompt)

In [0]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [0]:
chain.run("what is 2 + 2")
# Resulting in OutputParserException as Dolly is not able 
# to give the result in the proper JSON format